<a href="https://colab.research.google.com/github/GuilhermeOchoa/Data-Science/blob/master/Jogo_da_Velha_entre_IA's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

class Sucessor:
    def __init__(self, estado, valor, coluna=None):
        self.estado = estado
        self.valor = valor
        self.coluna = coluna

    def get_estado(self):
        return self.estado

    def get_valor(self):
        return self.valor

    def get_coluna(self):
        return self.coluna

    def __str__(self):
        return f"Estado: {self.estado}, Valor: {self.valor}, Coluna: {self.coluna}"

class Minimax:
    def __init__(self, estado):
        self.estado = estado

    def get_melhor(self):
        melhor_sucessor = self.algoritmo(self.estado, False, self.livres(self.estado))
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def get_melhor_ab(self):
        melhor_sucessor = self.algoritmo_ab(self.estado, False, self.livres(self.estado), -999, 999)
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def livres(self, estado):
        return estado.count('0')

    def jogada_aleatoria(self):
        posicoes_livres = [i for i, v in enumerate(self.estado) if v == '0']
        return random.choice(posicoes_livres) if posicoes_livres else None

    def gera_vizinhos(self, estado, caracter):
        vizinhos = []
        for i in range(9):
            if estado[i] == '0':
                novo_estado = estado[:]
                novo_estado[i] = caracter
                vizinhos.append((novo_estado, i))  # Retorna o estado e o índice da jogada
        return vizinhos

    def utilidade(self, atual, profundidade):
        if self.vencedor(atual, 'X'):
            return -1
        if self.vencedor(atual, 'O'):
            return 1
        if profundidade == 0:
            return 0
        return 100

    def vencedor(self, atual, caracter):
        # Verifica linhas, colunas e diagonais para vitória
        for i in range(3):
            if all(atual[i*3 + j] == caracter for j in range(3)) or all(atual[i + j*3] == caracter for j in range(3)):
                return True
        return (atual[0] == caracter and atual[4] == caracter and atual[8] == caracter) or \
               (atual[2] == caracter and atual[4] == caracter and atual[6] == caracter)

    def algoritmo(self, estado, jogador, profundidade):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, False, profundidade - 1)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, True, profundidade - 1)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
            return melhor_sucessor

    def algoritmo_ab(self, estado, jogador, profundidade, alfa, beta):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, False, profundidade - 1, alfa, beta)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
                if menor < alfa:
                    return melhor_sucessor
                beta = min(beta, menor)
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, True, profundidade - 1, alfa, beta)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
                if maior > beta:
                    return melhor_sucessor
                alfa = max(alfa, maior)
            return melhor_sucessor


In [ ]:
import numpy as np
import random

# Função de ativação sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.hidden_weights = None
        self.output_weights = None

    def inicializar_pesos(self, ga, inputs, target_outputs):
        # Inicializa os pesos usando o algoritmo genético
        melhor_rede = ga.run(inputs, target_outputs)
        self.hidden_weights = melhor_rede[0]
        self.output_weights = melhor_rede[1]

    def forward(self, inputs):
        inputs = np.append(inputs, 1)  # Adiciona o bias
        hidden_inputs = np.dot(self.hidden_weights, inputs)
        hidden_outputs = sigmoid(hidden_inputs)
        hidden_outputs = np.append(hidden_outputs, 1)  # Adiciona o bias
        final_inputs = np.dot(self.output_weights, hidden_outputs)
        final_outputs = sigmoid(final_inputs)
        return final_outputs

    def escolher_jogada(self, tabuleiro):
        inputs = np.array([1 if pos == 'O' else 0 for pos in tabuleiro])
        outputs = self.forward(inputs)
        melhor_jogada = np.argmax(outputs)
        return melhor_jogada

    def refinar_pesos(self, ga, resultado_partida, inputs):
        # Define o fitness baseado no resultado da partida
        fitness = 1 if resultado_partida == "vitória" else 0.5 if resultado_partida == "empate" else 0
        # Refinar a população de pesos com o algoritmo genético
        nova_populacao = ga.refinar_pesos_por_fitness([(self.hidden_weights, self.output_weights)], [fitness])
        self.hidden_weights, self.output_weights = nova_populacao[0]

In [ ]:
class GeneticAlgorithm:
    def __init__(self, input_size, hidden_size, output_size, population_size=50, generations=100, mutation_rate=0.1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            hidden_weights = np.random.uniform(-1, 1, (self.hidden_size, self.input_size + 1))
            output_weights = np.random.uniform(-1, 1, (self.output_size, self.hidden_size + 1))
            population.append((hidden_weights, output_weights))
        return population

    def crossover(self, parent1, parent2):
        child_hidden_weights = (parent1[0] + parent2[0]) / 2
        child_output_weights = (parent1[1] + parent2[1]) / 2
        return child_hidden_weights, child_output_weights


    def mutate(self, network):
        for weight_matrix in [network.hidden_weights, network.output_weights]:
            mutation_mask = np.random.rand(*weight_matrix.shape) < self.mutation_rate
            weight_matrix += mutation_mask * np.random.uniform(-0.5, 0.5, weight_matrix.shape)

    def run(self, inputs, target_outputs):
        population = self.initialize_population()
        for generation in range(self.generations):
            fitness_scores = [self.fitness_function(network, inputs, target_outputs) for network in population]
            sorted_population = sorted(zip(fitness_scores, population), key=lambda x: x[0], reverse=True)
            self.population = [x[1] for x in sorted_population[:self.population_size // 2]]
            new_population = []
            while len(new_population) < self.population_size:
                parent1, parent2 = random.choice(self.population), random.choice(self.population)
                child_hidden, child_output = self.crossover(parent1, parent2)
                self.mutate([child_hidden, child_output])
                new_population.append((child_hidden, child_output))
            population = new_population
        return sorted_population[0][1]

    def fitness_function(self, network, inputs, target_outputs):
        predictions = sigmoid(np.dot(network[1], sigmoid(np.dot(network[0], np.append(inputs, 1)))))
        return -np.mean((predictions - target_outputs) ** 2)

    def refinar_pesos_por_fitness(self, population, fitness_scores):
        sorted_population = [network for _, network in sorted(zip(fitness_scores, population), reverse=True)]
        new_population = sorted_population[:self.population_size // 2]
        while len(new_population) < self.population_size:
            parent1, parent2 = random.choice(new_population), random.choice(new_population)
            child1, child2 = self.crossover(parent1, parent2)
            self.mutate(child1)
            self.mutate(child2)
            new_population.extend([child1, child2])
        return new_population

In [ ]:
import random
import numpy as np

def inicializar_tabuleiro():
    return ['0' for _ in range(9)]

def imprimir_tabuleiro(tabuleiro):
    print(f"{tabuleiro[0]} | {tabuleiro[1]} | {tabuleiro[2]}")
    print("-" * 10)
    print(f"{tabuleiro[3]} | {tabuleiro[4]} | {tabuleiro[5]}")
    print("-" * 10)
    print(f"{tabuleiro[6]} | {tabuleiro[7]} | {tabuleiro[8]}")

def verificar_estado(tabuleiro):
    jogo = Minimax(tabuleiro)
    if jogo.vencedor(tabuleiro, 'X'):
        return 1
    elif jogo.vencedor(tabuleiro, 'O'):
        return 2
    elif '0' not in tabuleiro:
        return 3
    return 0

def jogada_ia_minimax(tabuleiro,dificuldade):
    jogo_ia = Minimax(tabuleiro)

    if dificuldade == 1:  # Nível 1: 25% Minimax, 75% Aleatório
        probabilidade_minimax = 0.25
    elif dificuldade == 2:  # Nível 2: 50% Minimax, 50% Aleatório
        probabilidade_minimax = 0.5
    else:  # Nível 3: 100% Minimax
        probabilidade_minimax = 1.0

    if random.random() < probabilidade_minimax:
        movimento_ia = jogo_ia.get_melhor()  # Pega o índice da jogada
    else:
        movimento_ia = jogo_ia.jogada_aleatoria()

    tabuleiro[movimento_ia] = 'X'
    print(f"IA Minimax escolheu a posição {movimento_ia}")


def jogada_ia_mlp(tabuleiro, mlp):
    # Transforma o tabuleiro em uma entrada adequada para a MLP
    inputs = np.array([1 if pos == 'O' else 0 for pos in tabuleiro])
    movimento_ia = mlp.escolher_jogada(tabuleiro)
    tabuleiro[movimento_ia] = 'O'
    print(f"IA MLP escolheu a posição {movimento_ia}")

    # Refinar os pesos da MLP após a jogada
    target_outputs = np.zeros(9)
    target_outputs[movimento_ia] = 1  # Marca a posição escolhida como a saída desejada
    mlp.refinar_pesos(inputs, target_outputs)

def jogar():
    num_partidas = int(input("Quantas partidas deseja simular? "))
    dificuldade =  int(input("Escolha a dificuldade (1: Fácil, 2: Médio, 3: Difícil): "))

    # Contadores para vitórias e empates
    vitorias_mlp = 0
    vitorias_minimax = 0
    empates = 0

    # Configurações da MLP
    INPUT_SIZE = 9
    HIDDEN_SIZE = 9
    OUTPUT_SIZE = 9
    # Criando a instância de MLP e GeneticAlgorithm
    mlp = MLP(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
    ga = GeneticAlgorithm(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

    # Inicializando pesos com o GA
    mlp.inicializar_pesos(ga, inputs=np.zeros(9), target_outputs=np.zeros(9))

    # Simulação das partidas

    for partida in range(num_partidas):
        print(f"\n--- Partida {partida + 1} ---")
        tabuleiro = inicializar_tabuleiro()
        jogadas = 0

        while True:
            imprimir_tabuleiro(tabuleiro)

            if jogadas % 2 == 0:  # Turno da MLP
                jogada_ia_mlp(tabuleiro, mlp)
            else:  # Turno da Minimax
                jogada_ia_minimax(tabuleiro,dificuldade)

            estado = verificar_estado(tabuleiro)
            if estado == 1:
                imprimir_tabuleiro(tabuleiro)
                print("IA Minimax venceu!")
                vitorias_minimax += 1
                break
            elif estado == 2:
                imprimir_tabuleiro(tabuleiro)
                print("IA MLP venceu!")
                vitorias_mlp += 1
                break
            elif estado == 3:
                imprimir_tabuleiro(tabuleiro)
                print("Empate!")
                empates += 1
                break

            jogadas += 1

        imprimir_tabuleiro(tabuleiro)

    # Exibe o resumo das partidas
    print("\n--- Resultado Final ---")
    print(f"Vitórias da IA MLP: {vitorias_mlp}")
    print(f"Vitórias da IA Minimax: {vitorias_minimax}")
    print(f"Empates: {empates}")

# Executa a simulação
jogar()


Quantas partidas deseja simular? 10
Escolha a dificuldade (1: Fácil, 2: Médio, 3: Difícil): 3


ValueError: shapes (9,10) and (9,) not aligned: 10 (dim 1) != 9 (dim 0)

In [ ]:
hidden_weights = np.random.uniform(-1, 1, (9, 9 + 1))
output_weights = np.random.uniform(-1, 1, (9, 9 + 1))
print(f"Pesos ocultos: {hidden_weights}")
print(f"Shape: {hidden_weights.shape}")
print(f"Pesos de saída: {output_weights}")


tabuleiro=['0' for _ in range(9)]
print(f"tabuleiro: {tabuleiro}")
inputs=np.array([1 if pos == 'O' else 0 for pos in tabuleiro])
print(f"inputs: {inputs}")

def forward(hidden_weights,output_weights, inputs):
        inputs = np.append(inputs, 1)  # Adiciona o bias
        print(f"inputs com bias: {inputs}")
        print(f"Shape: {inputs.shape}")
        hidden_inputs = np.dot(hidden_weights, inputs)
        print(f"hidden_inputs: {hidden_inputs}")
        hidden_outputs = sigmoid(hidden_inputs)
        hidden_outputs = np.append(hidden_outputs, 1)  # Adiciona o bias
        print(f"hidden_outputs - após a aplicação da função de transferência + Bias: {hidden_outputs}")
        final_inputs = np.dot(output_weights, hidden_outputs)
        print(f"final_inputs: {final_inputs}")
        final_outputs = sigmoid(final_inputs)
        print(f"final_outputs após aplicação da função de transferência: {final_outputs}")
        return final_outputs

outputs = forward(hidden_weights, output_weights, inputs)
print(f"outputs: {outputs}")

print(f"Melhor jogada  {np.argmax(outputs)}")


Pesos ocultos: [[-0.20455148  0.11574352 -0.8605303   0.36933034  0.88096577  0.64665583
  -0.7918297  -0.70120811  0.02761421  0.88001994]
 [-0.74379266  0.73574261  0.60792507 -0.09632823 -0.42603261 -0.02095879
   0.08589818 -0.14039949  0.7757328   0.09416234]
 [-0.66938287  0.23679956  0.99181111 -0.41026328 -0.02752732  0.43848777
   0.58525935  0.95045695 -0.33856716  0.06420307]
 [-0.323899   -0.83851302  0.3593158  -0.71047621 -0.58438544 -0.15098459
  -0.38494105  0.99647616  0.71504417  0.78023174]
 [ 0.61569096  0.00246179 -0.90226182 -0.69027038  0.03501077 -0.75082481
   0.96119752 -0.18068843  0.97048283  0.90216579]
 [ 0.06045002 -0.83103427 -0.7746111  -0.19094615 -0.93166038 -0.73210007
   0.35078916 -0.19447911  0.65532537 -0.17866344]
 [-0.33615971 -0.53334771  0.61885577  0.52395797 -0.21307275  0.54414815
   0.987524    0.28316326  0.76689568  0.81631823]
 [-0.53623604  0.32704631  0.06657317  0.70239409  0.85109707 -0.63134154
  -0.42213331 -0.49449728 -0.2046475